In [52]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.master('local[2]').getOrCreate()

In [53]:


dailytimespentwiththeinternetpercapitaworldwide = spark.read.csv("hdfs://hdfs-nn:9000/user/Projeto TABD/Bronze/DailyTimeSpentWithTheInternetPerCapitaWorldwide.csv", 
                                                 header=True, 
                                                 sep=";")


In [54]:
dailytimespentwiththeinternetpercapitaworldwide.printSchema()
dailytimespentwiththeinternetpercapitaworldwide.show()
dailytimespentwiththeinternetpercapitaworldwide.toPandas()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

+-----+-------+------+
|  _c0|    _c1|   _c2|
+-----+-------+------+
| null|   null|  null|
| null|Desktop|Mobile|
| 2011|     43|    32|
| 2012|     47|    36|
| 2013|     49|    45|
| 2014|     47|    63|
| 2015|     42|    81|
| 2016|     40|    94|
| 2017|     42|   109|
| 2018|     39|   122|
|2019*|     39|   132|
|2020*|     38|   143|
|2021*|     37|   155|
+-----+-------+------+



,_c0,_c1,_c2
0,None,None,None
1,None,Desktop,Mobile
2,2011,43,32
3,2012,47,36
4,2013,49,45
5,2014,47,63
6,2015,42,81
7,2016,40,94
8,2017,42,109
9,2018,39,122


In [55]:
dailytimespentwiththeinternetpercapitaworldwide = dailytimespentwiththeinternetpercapitaworldwide.selectExpr("_c0 as year", 
                                                                             "_c1 as desktop", 
                                                                             "_c2 as mobile")

In [56]:
dailytimespentwiththeinternetpercapitaworldwide.toPandas()

,year,desktop,mobile
0,None,None,None
1,None,Desktop,Mobile
2,2011,43,32
3,2012,47,36
4,2013,49,45
5,2014,47,63
6,2015,42,81
7,2016,40,94
8,2017,42,109
9,2018,39,122


In [57]:
dailytimespentwiththeinternetpercapitaworldwide = dailytimespentwiththeinternetpercapitaworldwide.withColumn(
    "year",
    f.regexp_replace(col("year"),"[*]", ""))

In [58]:
dailytimespentwiththeinternetpercapitaworldwide.toPandas()

,year,desktop,mobile
0,None,None,None
1,None,Desktop,Mobile
2,2011,43,32
3,2012,47,36
4,2013,49,45
5,2014,47,63
6,2015,42,81
7,2016,40,94
8,2017,42,109
9,2018,39,122


In [59]:
dailytimespentwiththeinternetpercapitaworldwide = dailytimespentwiththeinternetpercapitaworldwide.filter((dailytimespentwiththeinternetpercapitaworldwide.year != 'None'))

In [60]:
dailytimespentwiththeinternetpercapitaworldwide.toPandas()

,year,desktop,mobile
0,2011,43,32
1,2012,47,36
2,2013,49,45
3,2014,47,63
4,2015,42,81
5,2016,40,94
6,2017,42,109
7,2018,39,122
8,2019,39,132
9,2020,38,143


In [61]:
# Unpivot - Columns to rows
dailytimespentwiththeinternetpercapitaworldwide = dailytimespentwiththeinternetpercapitaworldwide.select(
        col("year"),
        expr("stack(2, 'desktop', `desktop`, 'mobile', `mobile`) as (device,time_spend)")
)

In [62]:
dailytimespentwiththeinternetpercapitaworldwide.toPandas()

,year,device,time_spend
0,2011,desktop,43
1,2011,mobile,32
2,2012,desktop,47
3,2012,mobile,36
4,2013,desktop,49
5,2013,mobile,45
6,2014,desktop,47
7,2014,mobile,63
8,2015,desktop,42
9,2015,mobile,81


In [63]:
dailytimespentwiththeinternetpercapitaworldwide = dailytimespentwiththeinternetpercapitaworldwide.withColumn(
    "year",
    dailytimespentwiththeinternetpercapitaworldwide["year"].cast(IntegerType()))

dailytimespentwiththeinternetpercapitaworldwide = dailytimespentwiththeinternetpercapitaworldwide.withColumn(
    "time_spend",
    dailytimespentwiththeinternetpercapitaworldwide["time_spend"].cast(IntegerType()))

In [64]:
dailytimespentwiththeinternetpercapitaworldwide.toPandas()

,year,device,time_spend
0,2011,desktop,43
1,2011,mobile,32
2,2012,desktop,47
3,2012,mobile,36
4,2013,desktop,49
5,2013,mobile,45
6,2014,desktop,47
7,2014,mobile,63
8,2015,desktop,42
9,2015,mobile,81


In [65]:
dailytimespentwiththeinternetpercapitaworldwide.printSchema()

root
 |-- year: integer (nullable = true)
 |-- device: string (nullable = true)
 |-- time_spend: integer (nullable = true)



In [66]:
dailytimespentwiththeinternetpercapitaworldwide \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/warehouse/tabd.db/DailyInternetUsageWorldwideByDevice/")